In [1]:
# import import_ipynb
# from algoritmos import *
from ipynb.fs.full.hormigas import *
import csv

ImportError: Could not import C:\Users\victo\Desktop\AlgoritmosGeneticosTSP\hormigas.ipynb for ipynb.fs.full.hormigas: not a valid ipynb file

In [ ]:
%matplotlib notebook
%matplotlib inline
fig, ax = plt.subplots(1,1)
plt.figure(figsize=(12,12)) 
ax.set_xlabel('Iteraciones') ; ax.set_ylabel('Coste mejor hormiga')

algoritmo = "Sistema de Hormigas"
print(algoritmo)
fichero = "st70"
nodos = leeFichero(fichero)
num_nodos = len(nodos)    # Fichero
distancias = getMDistancias(nodos)
semilla = random.randint(0,9999999)

# PARAMETROS DEL ALGORITMO
num_hormigas = 10
num_iteraciones = 100
feromona_ini = 1 / (num_nodos * getCosteCamino(algoritmoGreedy(distancias), distancias))
alpha = 1
beta = 2
evaporacion = 0.1

# Iniciamos la heuristica 1/d
heuristica = []
for row in range(len(distancias)):
    heuristica.append([1/(x+0.01) for x in distancias[row]])

t = time.time()

with Pool(processes=6) as p:
       resultados = p.starmap(algoritmo_SH, product([semilla], [num_nodos], [distancias], [num_hormigas], [feromona_ini], [heuristica], [alpha], [beta], [evaporacion]))[0]


#resultados = algoritmo_SH(semilla, num_nodos, distancias, num_hormigas, num_iteraciones, feromona_ini, heuristica, alpha, beta, evaporacion)

elapsed = time.time() - t
print("Tiempo de ejecución: %f\n" % elapsed )

coste = resultados[0].coste

print("Mejor sol Coste: %s \t Semilla: %s" % (coste, semilla) ) 

xs = resultados[2]
ys = resultados[3]

plt_dynamic(fig, xs, ys, ax)


titulo = algoritmo + " - Mejor solución " + fichero + ": " + str(coste)
pintaCamino(resultados[0].camino, nodos, titulo)

Sistema de Hormigas
